In [ ]:
import pandas as pd
import numpy as np

#LOADING THE DATA
df=pd.read_csv(r"C:\Users\riya choudhary\OneDrive\Desktop\Python dataset\smart_city_data.csv")

#PREVIEW THE DATA
df.info()
print('\n',df.head())
print("\nShape of the data (rows, columns):",df.shape)
print("\nData types of columns:\n",df.dtypes)
print("\nDimension of the dataset:",df.ndim)

#SUMMARY STATISTICS
print("\nSummary Statistics:\n",df.describe())

#CHECKING FOR MISSING VALUES
print("\nMissing values:")
print(df.isnull().sum())

#CHECKING FOR DUPLICATES
print("\nDuplicate values:")
print(df.duplicated().sum())

#EXPLORING CATEGORICAL COLUMNS
# Unique values and frequency for each categorical column
print("\nTop Cities with Most Records:\n", df['City_State'].value_counts().head(10))
print("\nUnique Cities:", df['City_State'].nunique())

print("\nTraffic Incident Types:\n", df['Traffic_Incident'].value_counts())
print("\nUnique Traffic Incidents:", df['Traffic_Incident'].nunique())

print("\nWeather Conditions:\n", df['Weather_Condition'].value_counts())
print("\nUnique Weather Conditions:", df['Weather_Condition'].nunique())
